In [72]:
import os
import re
import random
from typing import List, Dict, Any, Optional
from collections import defaultdict
from time import time
from glob import glob
import numpy as np
import gym
from textworld import EnvInfos
import textworld.gym
import spacy

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load("en_core_web_sm")

#import torch

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
gamefiles = []
GAMES_PATH = "./sample_games/"
for file in os.listdir(GAMES_PATH):
    if file.endswith('.ulx'):
        gamefiles.append(GAMES_PATH + file)

In [3]:
def play(agent, path, max_step=50, nb_episodes=10, verbose=True):
    request_infos = agent.select_additional_infos
    request_infos.max_score = True  # Needed to normalize the scores.

    gamefiles = [path]
    if os.path.isdir(path):
        gamefiles = glob(os.path.join(path, "*.ulx"))

    env_id = textworld.gym.register_games(gamefiles,
                                          request_infos=request_infos,
                                          max_episode_steps=max_step)
    env = gym.make(env_id)  # Create a Gym environment to play the text game.
    if verbose:
        if os.path.isdir(path):
            print(os.path.dirname(path), end="")
        else:
            print(os.path.basename(path), end="")

    # Collect some statistics: nb_steps, final reward.
    avg_moves, avg_scores, avg_norm_scores = [], [], []
    for no_episode in range(nb_episodes):
        obs, infos = env.reset()  # Start new episode.

        score = 0
        done = False
        nb_moves = 0
        while not done:
            command = agent.act(obs, score, done, infos)
            obs, score, done, infos = env.step(command)
            nb_moves += 1

        # Let the agent know the game is done.
        agent.act(obs, score, done, infos)

        if verbose:
            print(".", end="")
        avg_moves.append(nb_moves)
        avg_scores.append(score)
        avg_norm_scores.append(score / infos["max_score"])

    env.close()
    msg = "  \tavg. steps: {:5.1f}; avg. score: {:4.1f} / {}."
    if verbose:
        if os.path.isdir(path):
            print(msg.format(np.mean(avg_moves), np.mean(avg_norm_scores), 1))
        else:
            print(msg.format(np.mean(avg_moves), np.mean(
                avg_scores), infos["max_score"]))

In [106]:
class CustomAgent():
    
    def __init__(self) -> None:
        self._initialized = False
        self._epsiode_has_started = False
        self.mode = "test"
        self.command_history = []
        self.history_counter = 0
        
        # read in vocab.txt and map to id
        with open("./vocab.txt") as f:
            self.word_vocab = f.read().split("\n")
        self.word2id = {}
        self.id2word = []
        for i, w in enumerate(self.word_vocab):
            self.word2id[w] = i
            self.id2word.append(w)
        
        self.qvalues = [0] * len(self.id2word)

    def train(self) -> None:
        self.mode = "train"
        self.transitions = []
        self.last_score = 0
        self.no_train_step = 0
        self.moves = 0
        self.command_history = []
        self.history_counter = 0

    def test(self) -> None:
        self.mode = "test"

    @property
    def select_additional_infos(self) -> EnvInfos:
        """
        Returns what additional information should be made available at each game step.

        Requested information will be included within the `infos` dictionary
        passed to `CustomAgent.act()`. To request specific information, create a
        :py:class:`textworld.EnvInfos <textworld.envs.wrappers.filter.EnvInfos>`
        and set the appropriate attributes to `True`. The possible choices are:

        * `description`: text description of the current room, i.e. output of the `look` command;
        * `inventory`: text listing of the player's inventory, i.e. output of the `inventory` command;
        * `max_score`: maximum reachable score of the game;
        * `objective`: objective of the game described in text;
        * `entities`: names of all entities in the game;
        * `verbs`: verbs understood by the the game;
        * `command_templates`: templates for commands understood by the the game;
        * `admissible_commands`: all commands relevant to the current state;

        In addition to the standard information, game specific information
        can be requested by appending corresponding strings to the `extras`
        attribute. For this competition, the possible extras are:

        * `'recipe'`: description of the cookbook;
        * `'walkthrough'`: one possible solution to the game (not guaranteed to be optimal);

        Example:
            Here is an example of how to request information and retrieve it.

            >>> from textworld import EnvInfos
            >>> request_infos = EnvInfos(description=True, inventory=True, extras=["recipe"])
            ...
            >>> env = gym.make(env_id)
            >>> ob, infos = env.reset()
            >>> print(infos["description"])
            >>> print(infos["inventory"])
            >>> print(infos["extra.recipe"])

            Handicap is defined as follows
                max_score, has_won, has_lost,               # Handicap 0
                description, inventory, verbs, objective,   # Handicap 1
                command_templates,                          # Handicap 2
                entities,                                   # Handicap 3
                extras=["recipe"],                          # Handicap 4
                admissible_commands,                        # Handicap 5
        """
        
        return EnvInfos(description=True, inventory=True, extras=["recipe"], max_score = True, 
                        admissible_commands=True, has_won=True, has_lost=True)
      

    def _init(self) -> None:
        """ Initialize the agent. """
        self._initialized = True

        # [You can insert code here.]
    
    def _get_word_id(self, word):
        """
        look up id of a word in dictionary & add word to dictionaries if unknown.
        """
        if word not in self.word2id:        
            self.id2word.append(word)
            self.qvalues.append(0)
            self.word2id[word] = len(self.word2id)
        return self.word2id[word]

    
    def _get_word_by_id(self, id):
        """
        look up word by id in dictionary.
        returns None, if id does not exist.
        """
        if id < len(self.id2word):
            return self.id2word[id]
        else:
            return None
    
    def _get_qvalue(self, id):
        """
        Getter for qvalues (admissible commands with scores).
        """
        if id < len(self.qvalues):
            return self.qvalues[id]
        else:
            return None
    
    def _tokenize(self, text):
        """
        Simple tokenizer: strip out all non-alphabetic characters.
        """
        text = re.sub("[^a-zA-Z0-9\- ]", " ", text)
        word_ids = list(map(self._get_word_id, text.split()))
        return word_ids
    
    def _start_episode(self, obs: List[str], infos: Dict[str, List[Any]]) -> None:
        """
        Prepare the agent for the upcoming episode.
        Arguments:
            obs: Initial feedback for each game.
            infos: Additional information for each game.
        """
        if not self._initialized:
            self._init()

        self._epsiode_has_started = True

        # [You can insert code here.]

    def _end_episode(self, obs: List[str], scores: List[int], infos: Dict[str, List[Any]]) -> None:
        """
        Tell the agent the episode has terminated.
        Arguments:
            obs: Previous command's feedback for each game.
            score: The score obtained so far for each game.
            infos: Additional information for each game.
        """
        self._epsiode_has_started = False

        # [You can insert code here.]
    
    def update_qvalues(self, command : List[int],reward):
        """
        Updates admissible commands with additionally rewards.
        """
        for id in command:
            self.qvalues[id] += reward
    
    def calc_avg_qvalue(self, command) -> int:
        """
        Gets the average of all qvalues as a sum.
        """
        sum = 0
        for id in command:
            sum += self._get_qvalue(id)
        sum = sum/ len(command)
        return sum

    def choose_best_cmd(self,commands) -> int:
        """
        Gets randomly one of the best rewarded admissible commands.
        """
        avg_qvalues =  []
        for cmd in commands:
            avg_qvalues.append(self.calc_avg_qvalue(cmd))
        max_value = max(avg_qvalues)
        max_commands = []
        for i in range(len(commands)):
            if(max_value == avg_qvalues[i]):
                max_commands.append(i)
        return random.choice(max_commands)
    
    def get_alternative_cmd(self, commands) -> int:
        """
        Idea of an alternative command, if the current is used in the last 3 commands.
        Chooses the second best command.
        """
        avg_qvalues =  []
        for cmd in commands:
            avg_qvalues.append(self.calc_avg_qvalue(cmd))
     
        first_max = max(avg_qvalues[0],avg_qvalues[1]) 
        second_max = min(avg_qvalues[0],avg_qvalues[1]) 

        for i in range(2,len(avg_qvalues)): 
            if avg_qvalues[i] > first_max: 
                second_max = first_max
                first_max=avg_qvalues[i] 
            else: 
                if avg_qvalues[i]>second_max: 
                    second_max=avg_qvalues[i] 

        print("Second highest number is : ",str(second_max)) 
        return avg_qvalues.index(second_max)   
    
    def check_last_3_usages(self, cmds, infos, action):
        """
        building a historylist of commands to get out of a loop of death o_o
        """
        if self.command_history is not None:
            last_commands = self.command_history[-3:]
            
        # compare-counters for comparing actions from the 3 last moves.
        if self.history_counter > self.moves:
            same_command_in_list = True 
            counter_use_same_command = 0
            """ 
            If the current admissible command (action) is one of the last 3 commands, the while-loop
            will be skipped. 
            If the same random chosen command is going 2 times through the while-loop, after tahat, the loops
            will be skipped, too.
            """
            while same_command_in_list is True and counter_use_same_command < 3:
                try:
                    if last_commands.index(action):
                        best_index = self.get_alternative_cmd(cmds)
                        action = infos[best_index]
                        counter_use_same_command += 1
                    else:
                        same_command_in_list = False
                except ValueError:
                    same_command_in_list = False
                    print("last_commands List does not contain value")
                    
        # adds valid action to the command-history
        self.command_history.append(action)
        return last_commands
   
    def compare_action_recipe(self, action, recipe):
        """
        Comparing current action with recipe to add some scores.
        """
        noun, verb = self.slice_action_in_wordtypes(action)
        if noun in recipe:
            print("found noun in recipe: ", noun)
            # todo: add reward 
            
        # check if verb is in recipe is not necessary, 
        # because these kind of verbs like cut will never be found in a recipe.
        # todo: we need to find semantic connections.
        
        
    def slice_action_in_wordtypes(self, action):
        """
        slices action (admissible command) in verb and nouns.
        """
        #print("action: ", action)
        act = nlp(action)
        # noun as list, because in some action where two nouns, like put pepper in >>fridge<<
        # we need noun one with characteristics.
        noun_list = []
        verb, noun = "", ""
        
        for chunk in act.noun_chunks:
            #print("noun with characteristics: ", chunk.text)
            if chunk is not None:
                noun_list.append(chunk.text)
                noun = noun_list[0]

        for token in act:
            if token.pos_ == "VERB":
                verb = str(token)
                #print("verb: ", verb)
        
        if verb in noun:
            noun.replace(verb, "")
        #print("noun: ", noun)
        return noun, verb
        

    def act(self, obs: str, score: int, done: bool, 
            infos: Dict[str, List[Any]]) -> Optional[List[str]]:
        """
        Acts upon the current list of observations.

        One text command must be returned for each observation.

        Arguments:
            obs: Previous command's feedback for each game.
            scores: The score obtained so far for each game.
            dones: Whether a game is finished.
            infos: Additional information for each game.

        Returns:
            Text commands to be performed (one per observation).
            If episode had ended (e.g. `all(dones)`), the returned
            value is ignored.

        Notes:
            Commands returned for games marked as `done` have no effect.
            The states for finished games are simply copy over until all
            games are done.
        """
        
        #if all(dones):
        #   self._end_episode(obs, scores, infos)
        #    return  # Nothing to return.

        if not self._epsiode_has_started:
            self._start_episode(obs, infos)
        
        #choose best command based on "qvalues"
        cmds = []
        for i in range(len(infos["admissible_commands"])):
            cmds.append(self._tokenize(infos["admissible_commands"][i]))
        best_index = self.choose_best_cmd(cmds)
        action = infos["admissible_commands"][best_index]
        
        recipe = infos["extra.recipe"]
        self.compare_action_recipe(action, recipe)
        
        # last_commands for debug information and find new action if one is used the last 3 times.
        last_commands = self.check_last_3_usages(cmds, infos["admissible_commands"], action)
        
        if self.mode == "test":
            return action
        
        #train mode, counter update
        self.no_train_step += 1 
        reward = 0
        self.moves += 1
        self.history_counter += 1
        
        #calculate rewards
        if self.transitions:
            reward = score - self.last_score
            self.last_score = score
            if infos["has_won"]:
                reward += 10
            if infos["has_lost"]:
                reward -= 10
            if reward == 0:
                reward -= 1
            self.update_qvalues(self.transitions[-1],reward)

        # Debug output
        if(self.no_train_step % 1 == 0):
            print("")
            print(infos["extra.recipe"]) # type string
            print("Last 3 admissible commands: ", last_commands)
            print("train step:" + str(self.no_train_step))
            print("last reward:" + str(reward))
            print("obs in act: " + obs)
            command = "last command: "
            if self.transitions:
                for id in self.transitions[-1]:
                    command += " " + self._get_word_by_id(id)
                print(command)
        
        #save last command
        self.transitions.append(cmds[best_index])
        
        if done:
            if(infos["has_won"]):
                print("-------- WON GAME ----------")
            else:
                print("-------- LOST GAME ----------")
            print(" ")
            self.last_score = 0
            self.moves = 0
            self.transitions = []
            self.command_history = []
            
        return action

In [107]:
agent = CustomAgent()
agent.train()
starttime = time()
for i in range(5):
    print("-------------------------------------")
    print("GAME: " + str(i+1))
    print("-------------------------------------")
    play(agent, gamefiles[5])
print("Trained in {:.2f} secs".format(time() - starttime))


-------------------------------------
GAME: 1
-------------------------------------
tw-cooking-recipe3+take3+cook+cut+drop-vNZGCrQVhoJdt7Nx.ulx

Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

Last 3 admissible commands:  []
train step:1
last reward:0
obs in act: 


                    ________  ________  __    __  ________        
                   |        \|        \|  \  |  \|        \       
                    \$$$$$$$$| $$$$$$$$| $$  | $$ \$$$$$$$$       
                      | $$   | $$__     \$$\/  $$   | $$          
                      | $$   | $$  \     >$$  $$    | $$          
                      | $$   | $$$$$    /  $$$$\    | $$          
                      | $$   | $$_____ |  $$ \$$\   | $$          
      



Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

Last 3 admissible commands:  ['examine white onion', 'take banana from counter', 'eat banana']
train step:15
last reward:-1
obs in act: You eat the banana. Not bad.


last command:  eat banana


Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

Last 3 admissible commands:  ['take banana from counter', 'eat banana', 'examine red hot pepper']
train step:16
last reward:-1
obs in act: You couldn't pay me to eat that red hot pepper.



l

found noun in recipe:  red apple


Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

Last 3 admissible commands:  ['prepare meal', 'prepare meal', 'prepare meal']
train step:38
last reward:-1
obs in act: The recipe requires a sliced fried yellow potato.


last command:  prepare meal


Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

Last 3 admissible commands:  ['prepare meal', 'prepare meal', 'eat red apple']
train step:39
last reward:-10
obs in act: You eat the red apple. Not bad.

Second highest number is :  -2.25
last_commands List does not contain value


Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

Last 3 admissible commands:  ['examine carrot', 'take banana from counter', 'cook banana with oven']
train step:50
last reward:-1
obs in act: You roasted the banana.



last command:  cook banana with oven
last_commands List does not contain value


Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

Last 3 admissible commands:  ['take banana from counter', 'c

last_commands List does not contain value


Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

Last 3 admissible commands:  ['take cookbook from table']
train step:68
last reward:-1
obs in act: You take the cookbook from the table.



last command:  take cookbook from table
last_commands List does not contain value


Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

Last 3 admissible commands:  ['take cookbook from table', 'drop cookbook']
train step:69
last reward:-1
obs in act: You 

...

last_commands List does not contain value


Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

Last 3 admissible commands:  ['look', 'close fridge', 'examine pork chop']
train step:2069
last reward:-1
obs in act: You close the fridge.


last command:  close fridge
Second highest number is :  -238.75
last_commands List does not contain value


Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

Last 3 admissible commands:  ['close fridge', 'examine pork chop', 'open fridge']
train step:2

last_commands List does not contain value


Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

Last 3 admissible commands:  ['look', 'take knife', 'take purple potato from stove']
train step:2092
last reward:-1
obs in act: You pick up the knife from the ground.



last command:  take knife
Second highest number is :  -243.0
last_commands List does not contain value


Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

Last 3 admissible commands:  ['take knife', 'take purple potato from 



Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

Last 3 admissible commands:  ['examine white onion', 'examine white onion', 'close fridge']
train step:2114
last reward:-1
obs in act: The white onion looks appetizing.



last command:  examine white onion
Second highest number is :  -245.8
last_commands List does not contain value


Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

Last 3 admissible commands:  ['examine white onion', 'close fridge', 'examine white onion']
train st

last_commands List does not contain value


Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

Last 3 admissible commands:  ['take purple potato from table', 'put purple potato on stove', 'close fridge']
train step:2126
last reward:-1
obs in act: You close the fridge.


last command:  close fridge
last_commands List does not contain value


Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

Last 3 admissible commands:  ['put purple potato on stove', 'close fridge', 'open fridge']
train

found noun in recipe:  meal
Second highest number is :  -178.33333333333334


Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

Last 3 admissible commands:  ['eat red apple', 'prepare meal', 'eat red apple']
train step:2139
last reward:-1
obs in act: The recipe requires a sliced fried yellow potato.


last command:  prepare meal
found noun in recipe:  meal


Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

Last 3 admissible commands:  ['prepare meal', 'eat red apple', 'eat red apple

In [20]:
agent.test()
play(agent, gamefiles[5])

tw-cooking-recipe3+take3+cook+cut+drop-vNZGCrQVhoJdt7Nx.ulxinfos:  
Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

infos:  
Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

infos:  
Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare me

infos:  
Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

infos:  
Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

infos:  
Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

infos:  
Recipe #1
---------
Gather all following ingre

infos:  
Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

infos:  
Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

infos:  
Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

infos:  
Recipe #1
---------
Gather all following ingre

infos:  
Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

infos:  
Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

infos:  
Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

infos:  
Recipe #1
---------
Gather all following ingre

infos:  
Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

infos:  
Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

infos:  
Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

infos:  
Recipe #1
---------
Gather all following ingre

...

infos:  
Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

infos:  
Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

infos:  
Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

infos:  
Recipe #1
---------
Gather all following ingre

infos:  
Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

infos:  
Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

infos:  
Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

infos:  
Recipe #1
---------
Gather all following ingre

infos:  
Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

infos:  
Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

infos:  
Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

infos:  
Recipe #1
---------
Gather all following ingre

infos:  
Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

infos:  
Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

infos:  
Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

infos:  
Recipe #1
---------
Gather all following ingre

infos:  
Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

infos:  
Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

infos:  
Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red apple
  red onion
  yellow potato

Directions:
  slice the red apple
  roast the red apple
  slice the red onion
  slice the yellow potato
  fry the yellow potato
  prepare meal

infos:  
Recipe #1
---------
Gather all following ingre

In [95]:
for i in range(len(agent.qvalues)):
    if(agent.qvalues[i] != 0 ):
        print(agent._get_word_by_id(i) + " : " + str(agent.qvalues[i]))

apple : -189
banana : -114
bell : -209
carrot : -46
chop : -140
close : -135
cook : -155
cookbook : -141
counter : -150
drop : -142
eat : -120
examine : -424
fridge : -333
from : -283
hot : -42
insert : -62
into : -62
knife : -142
look : -234
meal : -59
on : -155
onion : -140
open : -134
orange : -57
oven : -146
pepper : -251
pork : -140
potato : -225
prepare : -59
purple : -188
put : -155
red : -232
stove : -148
table : -148
take : -331
white : -140
with : -155
yellow : -188


In [21]:
agent.train()
starttime = time()
for i in range(5):
    print("-------------------------------------")
    print("GAME: " + str(i+1))
    print("-------------------------------------")
    play(agent, gamefiles[8])
print("Trained in {:.2f} secs".format(time() - starttime))

-------------------------------------
GAME: 1
-------------------------------------
tw-cooking-recipe3+cook+cut+drop+go12-B10qS58OTbe0T8Qk.ulxinfos:  
Recipe #1
---------
Gather all following ingredients and follow the directions to prepare this tasty meal.

Ingredients:
  red hot pepper
  salt
  yellow potato

Directions:
  slice the red hot pepper
  slice the yellow potato
  grill the yellow potato
  prepare meal




TypeError: must be str, not list

In [44]:
# versuche
wordids = agent._tokenize("cook a meal")
print(wordids)

print(agent._get_word_id("cook"))
print(agent._get_word_by_id(4664))

[4664, 785, 11449]
4664
cook


In [78]:
def play_render(agent,gamefile):
    requested_infos = agent.select_additional_infos
    env_id = textworld.gym.register_games([gamefile], requested_infos)

    env = gym.make(env_id)
    obs, infos = env.reset()

    env.render()  # Print the initial observation.

    score = 0
    done = False
    while not done:
        command = agent.act(obs,score,done,infos)
        ob, score, done, infos = env.step(command)
        env.render()

In [79]:
agent.test()
play_render(agent, gamefiles[5])




                    ________  ________  __    __  ________
                   |        \|        \|  \  |  \|        \
                    \$$$$$$$$| $$$$$$$$| $$  | $$ \$$$$$$$$
                      | $$   | $$__     \$$\/  $$   | $$
                      | $$   | $$  \     >$$  $$    | $$
                      | $$   | $$$$$    /  $$$$\    | $$
                      | $$   | $$_____ |  $$ \$$\   | $$
                      | $$   | $$     \| $$  | $$   | $$
                       \$$    \$$$$$$$$ \$$   \$$    \$$
              __       __   ______   _______   __        _______
             |  \  _  |  \ /      \ |       \ |  \      |       \
             | $$ / \ | $$|  $$$$$$\| $$$$$$$\| $$      | $$$$$$$\
             | $$/  $\| $$| $$  | $$| $$__| $$| $$      | $$  | $$
             | $$  $$$\ $$| $$  | $$| $$    $$| $$      | $$  | $$
             | $$ $$\$$\$$| $$  | $$| $$$$$$$\| $$      | $$  | $$
             | $$$$  \$$$$| $$__/ $$| $$  | $$| $$_____ | $$__/ $$
          